In [1]:
import math

import numpy as np
from PIL import Image

In [2]:
img = Image.open('./peppers.png')
img_array = np.asarray(img)

In [3]:
print(img_array.shape)
print(img_array[0][0][2])

(384, 512, 3)
64


In [4]:
class Pixel:
    r = 0
    g = 0
    b = 0
    x = -1
    y = -1
    centroidX = 0
    centroidY = 0
    blocked = False
    def __init__(self, r, g, b, x, y):
        self.r = int(r)
        self.g = int(g)
        self.b = int(b)
        self.x = int(x)
        self.y = int(y)
        self.centroidX = int(x)
        self.centroidY = int(y)
        self.blocked = False

    def rgbDistance(self, pixel):
        a = abs(self.r - pixel.r)
        b = abs(self.g - pixel.g)
        c = abs(self.b - pixel.b)
        return a+b+c

    def xyDistance(self, pixel):
        return abs(self.x - pixel.x) + abs(self.y - pixel.y)

In [17]:
points = []
for i in range(384):
    for j in range(512):
        pixel = Pixel(img_array[i][j][0],img_array[i][j][1],img_array[i][j][2],i,j)
        points.append(pixel)

In [18]:
rgb_bandwidth = 40
xy_bandwidth = 80

In [19]:
n = len(points)
for random_point in points:
    if not random_point.blocked:
        centroid = Pixel(random_point.r, random_point.g, random_point.b, random_point.x, random_point.y)
        while True:
            cluster = []
            for point in points:
                if not point.blocked and centroid.rgbDistance(point) < rgb_bandwidth and centroid.xyDistance(point) < xy_bandwidth:
                    cluster.append(point)
                    point.blocked = True

            if len(cluster) == 0:
                break
            newCenX, newCenY = 0, 0
            for i in cluster:
                newCenX += i.x
                newCenY += i.y
            newCenX //= len(cluster)
            newCenY //= len(cluster)

            centroid.x = newCenX
            centroid.y = newCenY

            for i in cluster:
                i.r = centroid.r
                i.g = centroid.g
                i.b = centroid.b
            if abs(newCenY - centroid.x) <= 2 and abs(newCenX - centroid.y) <= 2:
                break
            

In [20]:
segmented_img_array = np.copy(img_array)
for i in points:
    segmented_img_array[i.x][i.y][0] = i.r
    segmented_img_array[i.x][i.y][1] = i.g
    segmented_img_array[i.x][i.y][2] = i.b
segmented_img = Image.fromarray(segmented_img_array)
segmented_img.save('./cl40d80_2.png')